In [2]:
import methods as me
import numpy as np
import matplotlib.pyplot as plt
import itertools as it

from scipy.optimize import minimize, curve_fit
from scipy.signal import find_peaks
from scipy import fftpack
from scipy import signal
import time

#  dir of this repo
dir_home = '/home/rmentel/interpolating-orbits/Github/'
dir_params = dir_home + 'params/'
dir_data = dir_home + 'data/'

In [3]:
# Define needed constants and labels
# AU to m
AU_to_M = 1.496*10**11
# AU/y to m/s
AUpY_to_MpS = 4743.183
# solar mass to kg
SM_to_kg = 1.988435*10**30
# year to seconds
Year_to_s = 3.1557*10**7
# gravitational constant G[m^3/kg*s^2]
G = 6.67428*10**-11
# gravitational parameter mu
mu = 1.327137415503024*10**20
# number of bodies
n_b = 9
batchsize = 11

name_label = ['Sun', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune', 'Pluto']
oe_label = ['Semimajor Axis [AU]', 'Eccentricity', 'Inclination [Deg]', 'A. of periapsis [Rad]', 'L. of ascending node [Rad]']
sin_labels = ['Period [Years]', 'Amplitude', 'Phase', 'Offset']
planetperiods = np.array([0.6152, 1, 1.8808, 11.862, 29.447, 84.012, 164.789, 249.589])
ratios = [3/4, 1/4, 2/3, 1/3, 1/2, 1, 2/1, 3/1, 3/2, 4/3, 4/1]

In [ ]:
# Labels for each fit type
fit_param_labels = [['Offset', 'Slope'],['Period [Years]', 'Amplitude', 'Phase', 'Offset'],['Period (1) [Years]', 'Amplitude (1)', 'Phase (1)', 'Offset', 'Period (2) [Years]', 'Amplitude (2)', 'Phase (2)'],['Period (1) [Years]', 'Amplitude (1)', 'Phase (1)', 'Offset', 'Period (2) [Years]', 'Amplitude (2)', 'Phase (2)', 'Period (3) [Years]', 'Amplitude (3)', 'Phase (3)'],['Period (1) [Years]', 'Amplitude (1)', 'Phase (1)', 'Offset', 'Period (2) [Years]', 'Amplitude (2)', 'Phase (2)', 'Period (3) [Years]', 'Amplitude (3)', 'Phase (3)', 'Period (4) [Years]', 'Amplitude (4)', 'Phase (4)']]
# Number of params that are fitted for each fit type
fit_param_number = [2, 4, 7, 10, 13]

In [4]:
# Read in ORIGINAL Brutus data
file = open(dir_data + 'run_N9_solar_system_jpl_0_0_0_t65Myr_e14_Lw128.diag', 'r')
body_o = file.readlines()[:]
steps = 1000
# 1000 steps is enough to cover the entire new simulation
# steps = 182035

time_o = np.zeros((int(steps), 1))
cart_o = np.zeros((int(steps), n_b, 7))
kepl_o = np.zeros((int(steps), n_b, 7))

# Write array of cartesian parameters: Solar mass, AU, AU/Y
for i in range(int(steps)): # for all time steps ...
    batch = body_o[(batchsize*i):(batchsize*(i+1))]
    time_o[i, 0] = float((batch[0].split())[0])/6.28
    for j, line in enumerate(batch[1:-1]): # for all bodies...
        words = line.split()
        cart_o[i, j, :] = [float(x) for x in words]
        cart_o[i, j, 4:8] = 2*np.pi*cart_o[i, j, 4:8]

for i, step in enumerate(cart_o[:steps]):
    # Calculate COM:
    R_COM = np.zeros((1,3))
    V_COM = np.zeros((1,3))
    M = 0
    for j in range(8):
        m = SM_to_kg*step[j, 0]
        M = M + m
        R = AU_to_M*step[j, 1:4]
        V = AUpY_to_MpS*step[j, 4:7]
        R_COM = R_COM + m*R
        V_COM = V_COM + m*V
    r_COM = R_COM/M
    v_COM = V_COM/M   

#     Use for-loop for all bodeis
#     for j in range(9):
#     or specify one body with j
    j = 3
    m = SM_to_kg*step[j, 0]
    R = AU_to_M*step[j, 1:4]
    V = AUpY_to_MpS*step[j, 4:8]
    r = (R - r_COM)[0]
    v = (V - v_COM)[0]
    # a, e, i, om, Om, M, E
#     Read in the SMA
    kepl_o[i, j, 0] = me.CartesianToKepler(m, r, v)[0]/AU_to_M
#     Read in all OEl
#     Or specify one with k
#     0 - ecc, 1 - inc, 2 - aop, 3 - lan
#     for k in range(6):
    k = 0
    kepl_o[i, j, k+1] = me.CartesianToKepler(m, r, v)[k+2]

In [5]:
# Read in NEW Brutus data
# Works analogous to cell for original Brutus data

file = open(dir_data + 'brutus_venus_to_pluto_62800y_182d_e14_lw128.txt', 'r')
body_n = file.readlines()[:]
steps = 125500 # entire data
# steps = 1000

# Define arrays for cartesians and timeline
time_n = np.zeros((int(steps), 1))
cart_n = np.zeros((int(steps), n_b, 7))
kepl_n = np.zeros((int(steps), n_b, 7))

# Write array of cartesian parameters: Solar mass, AU, AU/Y
for i in range(int(steps)): # for all time steps ...
    batch = body_n[(batchsize*i):(batchsize*(i+1))]
    time_n[i, 0] = float(((batch[0]).split()[0]))
    for j, line in enumerate(batch[1:-1]): # for all bodies...
        words = line.split()
        cart_n[i, j, :] = [float(x) for x in words]
        cart_n[i, j, 4:8] = cart_n[i, j, 4:8]

for i, step in enumerate(cart_n[:steps]):
    # Calculate COM:
    R_COM = np.zeros((1,3))
    V_COM = np.zeros((1,3))
    M = 0
    for j in range(8):
        m = SM_to_kg*step[j, 0]
        M = M + m
        R = AU_to_M*step[j, 1:4]
        V = AUpY_to_MpS*step[j, 4:7]
        R_COM = R_COM + m*R
        V_COM = V_COM + m*V
    r_COM = R_COM/M
    v_COM = V_COM/M
#     SINGLE BODY
    j = 4
#     for j in range(9):
    m = SM_to_kg*step[j, 0]
    R = AU_to_M*step[j, 1:4]
    V = AUpY_to_MpS*step[j, 4:8]
    r = (R - r_COM)[0]
    v = (V - v_COM)[0]
# a, e, i, om, Om, M, E
#     SINGLE ELEMENT
    kepl_n[i, j, 0] = me.CartesianToKepler(m, r, v)[0]/AU_to_M
    k = 0
#    for k in range(6):
    kepl_n[i, j, k+1] = me.CartesianToKepler(m, r, v)[k+2]

In [6]:
# Sanity check
print (kepl_n[0, j,:])

[5.19916704 0.04893733 0.         0.         0.         0.
 0.        ]


In [7]:
# Get range of fits for all tau for specified body b and OEl c
b = 3
c = 1

# give fit type/number of sinusoids that are fitted to the data
fit_type = 2
n_params = fit_param_number[fit_type]
fit_labels = fit_param_labels[fit_type]

original = np.zeros((len(kepl_o), 2))
original[:,0] = time_o[:, 0]
original[:,1] = kepl_o[:, b, c]

new = np.zeros((len(kepl_n), 2))
new[:,0] = time_n[:, 0]
new[:,1] = kepl_n[:, b, c]

# Uncomment desired body/oel
# Use with or without bounds
# Else, use found parameters as init_vals, set bounds to +- 5%, then vary bounds until good fit achieved

# Venus SMA
# init_vals = [2.635, 3e-3, 0, 0.725, 2.5, 5.5e-3, 0, 2.178, 1.1e-2, 0]
# bounds = [[2.6, 2e-3, 0, 0.7, 2.4, 1e-3, 0, 2.15, 1e-2, 0], [2.7, 5e-3, 2*np.pi, 0.8, 2.6, 1e-2, 2*np.pi, 2.2, 2e-2, 2*np.pi]]

# Jupiter SMA
# init_vals = [11.8467, 5e-3, 0, 0, 60.657, 6e-4, 0]
# bounds = [[11.846, 1e-3, 0, 0, 60.65, 4e-4, 0], [11.8469, 9e-3, 2*np.pi, 2*np.pi, 60.66, 8e-4, 2*np.pi]]
# Jupiter ECC
# init_vals = [60.858, 7e-4, 0, 0, 11.8565, 3e-4, 0]
# bounds = [[60.853, 5e-4, 0, -1, 11.85, 2e-4, 0], [60.865, 1e-3, 2*np.pi, 1, 11.86, 5e-4, 2*np.pi]]
# Jupiter AOP
# init_vals = [60.6, 1e-2, 0, 1.5, 11.85, 5e-4, 0]
# bounds = [[60.6, 1e-3, 0, 0, 11.85, 1e-4, 0], [60.7, 5e-2, 2*np.pi, 2*np.pi, 11.87, 1e-3, 2*np.pi]]

# Saturn SMA
# init_vals = [29.4, 1e-2, 0, 0, 11.85, 1e-2, 0]
# bounds = [[29.3, 1e-3, 0, 0, 11.8, 1e-4, 0], [29.5, 1e-0, 2*np.pi, 20, 11.9, 1e-0, 2*np.pi]]
# Saturn ECC
# init_vals = [930, 1e-4, 0, 0.01, 29.4, 1e-3, 0]
# bounds = [[920, 1e-5, 0, 0, 29.35, 1e-5, 0], [940, 1e-1, 2*np.pi, 1, 29.45, 1e-1, 2*np.pi]]
# SATURN AOP
# init_vals = [29.45, 1e-2, 0, 0]
# bounds = [[29.4, 1e-3, 0, 0], [29.5, 5e-1, 2*np.pi, 2*np.pi]]

# URANUS SMA
# init_vals = [171.29, 4.5e-3, 0, 19.237, 84.01, 4e-3, 0]
# bounds = [[171.28, 4e-3, 0.0, 19.236, 84., 3e-3, 0], [171.31, 5e-3, 2*np.pi, 19.239, 84.1, 5e-3, 2*np.pi]] 
# URANUS AOP
# init_vals = [84, 5e-2, 0, 4.5]
# bounds = [[83.8, 1e-3, 0, 4], [84.2, 1e-1, 2*np.pi, 5]]

# Neptune SMA
# init_vals = [164, 1e-3, 0, 0]
# bounds = [[162, 5e-4, 0, -1], [168, 2.5e-3, 2*np.pi, 1]]
# NEPTUNE ECC
# init_vals = [164.5, 2.5e-3, 0, 0]
# bounds = [[164, 2e-3, 0, 0.0], [165, 3.5e-3, 2*np.pi, 0.1]]
# NEPTUNE AOP
# init_vals = [160, 1e-1, 0, 4]
# bounds = [[160, 5e-3, 0, 0], [170, 3e-1, 6.5, 2]]

# PLUTO SMA
# init_vals = [247, 2e-2, 0, 40]
# bounds = [[245, 1e-3, 0, 38], [250, 1e-1, 2*np.pi, 42]]
# PLUTO ECC
# init_vals = [247, 2e-2, 0, 0] # PLUTO ECC
# PLUTO AOP
# init_vals = [247, 2e-2, 0, 0] # PLUTO AOP

# this routine fits desired number of sinusoids to 2*n data points to 12,000 taus, spaced by 5yr
n = 2000
n_end = 12000
# n_end = 1000
maxfev = 10000
ftol = 1e-8

# Array with the results from the fits
arr_results = np.zeros((n_end, n_params+2))
# 0: tau of fit
# 1: r2-value
# 2: phase
# 3: amplitude
# 4: offset
# for more than one sine@
# 5: period 2
# 6: amp 2
# 7: phase 2
# 8: period 3
# 9: amp 3
# 10: phase 3

for j in range (n_end):
    i = 10*j
    tau = 0.5*i
    i_0, i_1 = me.Adj_I(i, n)
    data = new[i_0:i_1, :]
    x = data[:,0]
    y = data[:,1]

#     Uncomment type of fit

# LINEAR FIT
#     res, cov = curve_fit(me.xLin, x, y, p0=init_vals, maxfev=maxfev, ftol=ftol)
#     fit = me.Linear(x, res)

# ONE SINE
#     res, cov = curve_fit(me.xSine, x, y, p0=init_vals, maxfev=maxfev, ftol=ftol, bounds=bounds)
#     fit = abs(me.Sin(x, res))

# TWO SINES
    res, cov = curve_fit(me.xTwoAddedSines, x, y, p0=init_vals, maxfev=maxfev, ftol=ftol, bounds=bounds)
    fit = me.AddedSines(x, res)

# THREE SINES
#     res, cov = curve_fit(me.xThreeAddedSines, x, y, p0=init_vals, maxfev=maxfev, ftol=ftol)#, bounds=bounds)
#     fit = me.ThreeAddedSines(x, res)

    r2 = me.Goodness(Y=y, Fit=fit)
#     Specify if routine should use result from last iteration as init_vals for next one
    init_vals = res

#      arr_results[j,:] = tau, r2, res[0], res[1] # LINEAR
#     arr_results[j,:] = tau, r2, res[0], res[1], np.mod(res[2], 2*np.pi), res[3] # SINE
    arr_results[j,:] = tau, r2, res[0], res[1], np.mod(res[2], 2*np.pi), res[3], res[4], res[5], np.mod(res[6], 2*np.pi)
#     arr_results[j,:] = tau, r2, res[0], res[1], np.mod(res[2], 2*np.pi), res[3], res[4], res[5], np.mod(res[6], 2*np.pi), res[7], res[8], np.mod(res[9], 2*np.pi)

#     print (res)
print ('Done')

SyntaxError: invalid syntax (<ipython-input-7-1082f8388c89>, line 74)

In [ ]:
# Plotting the results
x = arr_results[:, 0]

x1 = 60000

fig = plt.figure(figsize=(21, n_params*7))
ax1 = fig.add_subplot(2+n_params,1,1)
ax2 = fig.add_subplot(2+n_params,1,2)
ax3 = fig.add_subplot(2+n_params,1,3)
ax4 = fig.add_subplot(2+n_params,1,4)
ax5 = fig.add_subplot(2+n_params,1,5)
ax6 = fig.add_subplot(2+n_params,1,6) 
# plus these for Two sines:
ax7 = fig.add_subplot(2+n_params,1,7)
ax8 = fig.add_subplot(2+n_params,1,8)
ax9 = fig.add_subplot(2+n_params,1,9)
# plus these for three sines:
# ax10 = fig.add_subplot(2+n_params,1,10)
# ax11 = fig.add_subplot(2+n_params,1,11)
# ax12 = fig.add_subplot(2+n_params,1,12)

# axes = [ax1, ax2, ax3, ax4] # LINEAR
# axes = [ax1, ax2, ax3, ax4, ax5, ax6] # SINE
axes = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9] # TWO SINE
# axes = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9, ax10, ax11, ax12] # THREE SINE


ax1.set_title('Fit parameters for ' + oe_label[c] + ' of ' + name_label[b], fontsize=32)
# ax1.scatter(original[:,0], original[:,1], s=10, c='k', zorder=30, marker='o', label='Original data')
ax1.scatter(new[:,0], new[:,1], s=1, c='r', label='New data')
ax1.set_xlabel('Time [Years]', fontsize=16)
ax1.set_ylabel(oe_label[c] + ' of ' + name_label[b], fontsize=24)
ax1.set_xlim(0, x1)
# ax1.set_ylim(ylims)
ax1.tick_params(labelsize=16)
ax1.legend(fontsize=16)


ax2.scatter(x, arr_results[:,1], c='k', s=1, label="Goodness of fit")
ax2.set_xlim(0, x1)
# ax2.set_ylim(0,1)
ax2.set_ylabel("R2-Goodness of Fit", fontsize=24)
ax2.set_xlabel('Tau [years]', fontsize=16)
ax2.tick_params(labelsize=16)
ax2.legend(fontsize=16)

# lims = [(245, 255), (2e-3, 3e-3), (0, 6.3), (0.24, 0.25)]

for i in range(n_params):
    ax = axes[i+2]
    y = arr_results[:, i+2]
    ax.scatter(x, abs(y), s=2, c='k', label='Fit parameter at given tau')
    ax.axhline(y=init_vals[i], c='r', lw=1, label='Initial Value for fit')
    ax.set_ylabel(fit_labels[i], fontsize=24)
    ax.set_xlabel('Tau [years]', fontsize=16)
    ax.set_xlim(0, x1)
#     ax.set_ylim(lims[i])
    ax.tick_params(labelsize=16)
    ax.legend(fontsize=16)

    plt.show()
plt.close()

In [ ]:
# Run a quick fit on (section of) data
# For example to fit long-term variatiomms

n0, n1 = 0, 120000

b, c = 4, 1

new = np.zeros((n1, 2))
new[:,0] = time_n[:n1, 0]
new[:,1] = kepl_n[:n1, b, c]

x = new[:,0]
y = new[:,1]
#x = arr_results[n0:n1,0]
#y = arr_results[n0:n1,2]

init_vals = [50000, 1e-2, 0, 171]
bounds = [[4200, 1e-3, 0, 170], [4400, 1e-1, 2*np.pi, 172]]

maxfev = 10000
ftol = 1e-8
res, cov = curve_fit(me.xSine, x, y, p0=init_vals, maxfev=maxfev, ftol=ftol)#, bounds=bounds)
# res, cov = curve_fit(me.xTwoAddedSines, x, y, p0=init_vals, maxfev=maxfev, ftol=ftol)#, bounds=bounds)
fit = abs(me.Sin(x, res))
# fit = abs(me.AddedSines(x, res))
r2 = me.Goodness(Y=y, Fit=fit)
print (r2)
print (res)

fig = plt.figure(figsize=(21,7))
ax = fig.add_subplot(111)

ax.scatter(x, y, s=1, c='k')#, label='Period X of fit to X')
ax.scatter(x, fit, s=1, c='r', label='Sine fit to progression')
# ax.scatter(x, me.Sin(x, ), s=1, c='b', label='fit?')

ax.set_ylabel("Period X of Fit", fontsize=16)
ax.set_xlabel("Time [Years]", fontsize=16)

plt.legend(fontsize=16)
plt.show()
plt.close()

In [ ]:
# Multiplot fitting
# Fits N sines to the data for one tau
# Fits all combos from set of different periods " a" as init_vals
# Use this to try out different combinations of periods
# a = [1666, 172, 84.7, 56.2, 31.1]
a = [90, 29.4, 11.8, 9.9, 1.9]
# a = [ 10.67, 2.23, 1.02]
i_s = [0,4,7,10]
N = 3
for i, x in enumerate(it.combinations(a, N)):
    x = x
print (i)

for p in it.combinations(a, N):
    print (p)
#     init_vals = [p[0], 1e-2, 0, 1., p[1], 1e-1, 0, p[2], 1e-1, 0, p[3], 1e-1, 0]
    init_vals = [p[0], 1e-1, 0, 1., p[1], 1e-1, 0, p[2], 1e-1, 0]
#     init_vals = [p[0], 1e-2, 0, 1., p[1], 1e-3, 0]
    tau = 1500
    n = 1000
    maxfev = 10000
    ftol = 1e-9
#     res = FitTwoAddedSines(new, tau, n, init_vals, maxfev)
    res = me.FitThreeAddedSines(new, tau, n, init_vals, maxfev)
#     res = FitFourAddedSines(new, tau, n, init_vals, maxfev)
    print (res)

    i_tau = int(2*tau)
    i_0, i_1 = me.Adj_I(i_tau, n)
    x = new[i_0:i_1, 0]
    y = new[i_0:i_1, 1]
    x2 = np.linspace(x[0], x[-1], len(x)*10)

    print (me.Goodness(Y=y, Fit=me.ThreeAddedSines(x, res)))
    
    fig = plt.figure(figsize=(21,7))
    ax = fig.add_subplot(111)
    ax.plot(x, y, ms=0, lw=1, ls='--', c='r', label='New data')
#     ax.plot(x2, AddedSines(x2, res), c='k', lw=1, zorder=20, label='Added Sine fit')
    ax.plot(x2, me.ThreeAddedSines(x2, res), c='k', lw=1, zorder=20, label='Added Sine fit')
#     ax.plot(x2, FourAddedSines(x2, res), c='k', lw=1, zorder=20, label='Added Sine fit')
    ax.axvline(x=tau, c='b', lw=1)
#     plt.xlim(tau-2*n, tau+2*n)
    plt.xlim(tau-500, tau+500)
    plt.xlabel('Time [Years]', fontsize=16)
    plt.ylabel(oe_label[c] + ' of ' + name_label[b], fontsize=16)
    plt.legend(fontsize=16)
    plt.title('Fit for ' + oe_label[c] + ' of ' + name_label[b], fontsize=20)
    plt.show()
    plt.close()
#     for i in i_s:
#         print (round(res[i],2), np.log10(abs(res[i+1])))
    print ('---')

# Detrending

In [ ]:
# Takes data and subtracts from that the offset from the fits
# Then, can fit the detrended progression

residual = np.zeros((n_end, 3))
residual[:,0] = time_n[0:n_end*10:10, 0]
# residual[:,1] = arr_results[:,5]
residual[:,1] = new[:n_end*10:10,1]-arr_results[:,5]

fig = plt.figure(figsize=(21,7))
ax = fig.add_subplot(111)

x = residual[:,0]
dy = 5e-2
# ax.scatter(x, new[:n_end*10:10,1], c='k', s=1)

ax.scatter(x, residual[:,1], c='r', s=1)
# ax.set_ylim(-dy, dy)
# ax.set_xlim(0, 1000)
plt.show()
plt.close()


In [8]:
# Fit and plot detrended data
# WARNING: may need some time!

# Get range of fits for residual

fit_type = 2
n_params = fit_param_number[fit_type]
fit_labels = fit_param_labels[fit_type]

# n0, n1 = 5000, 10000
n0,n1 = 0, 45000

# init_vals = [247, 2e-2, 0, 40] # PLUTO SMA
# bounds = [[245, 1e-3, 0, 38], [250, 1e-1, 2*np.pi, 42]] #18000, 1e-1, 0], , 22000, 5e-1, 6.29]] # PLUTO SMA
# init_vals = [164, 1e-3, 0, 0]
# bounds = [[162, 5e-4, 0, -1], [168, 2.5e-3, 2*np.pi, 1]] # NEPTUNE SMA
# OLD init_vals = [171.3, 5e-3, 0, 0, 84.01, 7e-3, 0] # URANUS SMA
# OLD bounds = [[171., 3e-3, 0.0, -1, 83.9, 5e-3, 0], [171.5, 6e-3, 2*np.pi, 1, 84.1, 8.5e-3, 2*np.pi]] # URANUS SMA
# init_vals = [171.3, 4.5e-3, 0, 0, 84.0, 4e-3, 0] # URANUS SMA
# bounds = [[171.25, 3e-3, 0.0, -1, 83.95, 2e-3, 0], [171.35, 6e-3, 2*np.pi, 1, 84.05, 6e-3, 2*np.pi]] # URANUS SMA
init_vals = [930, 1e-4, 0, 0.01, 29.47, 1e-3, 0]#, 30, 1e-3, 0] # SATURN ECC
bounds = [[920, 1e-6, 0, -1, 29.4, 1e-6, 0], [940, 1e-1, 2*np.pi, 1, 29.5, 1e-1, 2*np.pi]] # SATURN ECC

# init_vals = [84, 5e-2, 0, 0] # URANUS AOP
# bounds = [[83.8, 2e-2, 0, -1], [84.2, 1e-1, 2*np.pi, 1]] # URANUS AOP
# init_vals = [19.847, 7e-3, 0, 0]
# init_vals = [29.45, 5e-2, 0, 0]
# init_vals = [19.847, 7e-3, 0, 0, 12, 1e-5, 0]
# init_vals = [60.858, 7e-4, 0, 0, 11.8565, 3e-4, 0]
# init_vals = [164.7, 1.5e-3, 0, 0, 85.6, 1e-3, 0]
# init_vals = [172, 4.4e-3, 0, 0, 84.7, 5e-3, 0]
# init_vals = [150, 5e-2, 0, 0]#, 20000, 2e-1, 0]
# init_vals = [29.44, 1e-4, 0, 0.0, 11.980, 1e-3, 0]
# bounds = [0,np.inf]
# bounds = [[29.4, 1e-3, 0, -1], [29.5, 1e-1, 2*np.pi, 1]]
# bounds = [[60.853, 5e-4, 0, -1, 11.85, 2e-4, 0], [60.865, 1e-3, 2*np.pi, 1, 11.86, 5e-4, 2*np.pi]] # Earth ?
# bounds = [[60.853, 5e-4, 0, -1, 11.85, 2e-4, 0], [60.865, 1e-3, 2*np.pi, 1, 11.86, 5e-4, 2*np.pi]] # Jupiter ECC
# bounds = [[19.845, 5e-3, 0, -1], [19.848, 1e-2, 6.29, 1]] # Jupiter
# bounds = [[19.845, 5e-3, 0, -1, 11.99, 1e-6, 0], [19.848, 1e-2, 6.29, 1, 12.01, 1e-3, 6.29]] # Jupiter
# bounds = [[246, 4.7e-2, 0, -1], [251, 5.5e-2, 6.29, 1]] # PLUTO
# bounds = [[246, 1.0e-2, 0, 0], [248, 1.2e-2, 6.5, 1e-1]] # NEPTUNE
# bounds = [[170, 4e-3, 0, 0, 80, 4e-3, 0], [175, 5e-3, 6.3, 1e-1, 90, 5e-3, 6.3]] # URANUS
# bounds = [[29.4, 1e-3, 0, -1, 60.6, 1e-3, 0], [29.5, 1e-1, 6.3, 1, 60.9, 5e-2, 6.3]]


n = 1000
# n_end = 12000
# n_end = n1-n0
n_end = 45000
maxfev = 50000
ftol = 1e-8


arr_results_n = np.zeros((n_end, n_params+2))
# residual = np.zeros((n_end, 2))
# residual[:,0] = time_n[0:n_end*10:10, 0]
# residual[:,0] = time_n[n0:n1:n_end*10:10, 0]
# residual[:,1] = new[n_end*10:10,1]-arr_results[:n_end,5]
# residual[:,1] = new[n0:n1:n_end*10:10,1]-arr_results[:n_end,5]
# residual[:,1] = arr_results[:,-1]
# residual[:,1] = newdata[:n_end*10:10]
print (init_vals)

for j in range (n_end):
    i = 1*j
#     tau = 5*i
    tau = 0.5*i + 15000
    i_0, i_1 = me.Adj_I(i, n)
    data = residual[i_0:i_1, :]
    x = data[:,0]
    y = data[:,1]
#     res, cov = curve_fit(me.xSine, x, y, p0=init_vals, maxfev=maxfev, ftol=ftol, bounds=bounds)
    res, cov = curve_fit(me.xTwoAddedSines, x, y, p0=init_vals, maxfev=maxfev, ftol=ftol, bounds=bounds)
#     fit = abs(me.Sin(x, res))
    fit = abs(me.AddedSines(x, res))
    r2 = abs(me.Goodness(Y=y, Fit=fit))
#     arr_results[j,:] = tau, r2, res[0], res[1] # LINEAR
#     arr_results_n[j,:] = tau, r2, res[0], res[1], np.mod(res[2], 2*np.pi), res[3] # SINE
    arr_results_n[j,:] = tau, r2, res[0], res[1], np.mod(res[2], 2*np.pi), res[3], res[4], res[5], np.mod(res[6], 2*np.pi)
    init_vals = res
    
# Plotting res of residual fit
x = arr_results_n[:, 0]

# x1 = 65000
x1 = 55000

fig = plt.figure(figsize=(21, n_params*7))
ax1 = fig.add_subplot(2+n_params,1,1)
ax2 = fig.add_subplot(2+n_params,1,2)
ax3 = fig.add_subplot(2+n_params,1,3)
ax4 = fig.add_subplot(2+n_params,1,4)
ax5 = fig.add_subplot(2+n_params,1,5)
ax6 = fig.add_subplot(2+n_params,1,6)
ax7 = fig.add_subplot(2+n_params,1,7)
ax8 = fig.add_subplot(2+n_params,1,8)
ax9 = fig.add_subplot(2+n_params,1,9)

# axes = [ax1, ax2, ax3, ax4, ax5, ax6] # SINE
axes = [ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9] # TWO SINE

ax1.set_title('Fit parameters for ' + oe_label[c] + ' of ' + name_label[b], fontsize=32)
ax1.scatter(residual[:,0], residual[:,1], s=1, c='r', label='New data')
ax1.set_xlabel('Time [Years]', fontsize=16)
ax1.set_ylabel(oe_label[c] + ' of ' + name_label[b], fontsize=24)
ax1.set_xlim(0, x1)
ax1.tick_params(labelsize=16)
ax1.legend(fontsize=16)

ax2.scatter(x, arr_results_n[:,1], c='k', s=1, label="Goodness of fit")
ax2.set_xlim(0, x1)
ax2.set_ylabel("R2-Goodness of Fit", fontsize=24)
ax2.set_xlabel('Tau [years]', fontsize=16)
ax2.tick_params(labelsize=16)
ax2.legend(fontsize=16)

for i in range(n_params):
    ax = axes[i+2]
    y = arr_results_n[:, i+2]
    ax.scatter(x, abs(y), s=2, c='k', label='Fit parameter at given tau')
    ax.axhline(y=init_vals[i], c='r', lw=1, label='Initial Value for fit')
    ax.set_ylabel(fit_labels[i], fontsize=24)
    ax.set_xlabel('Tau [years]', fontsize=16)
    ax.set_xlim(0, x1)
#     ax.set_ylim(lims[i])
    ax.tick_params(labelsize=16)
    ax.legend(fontsize=16)
plt.show()
plt.close()

NameError: name 'fit_param_number' is not defined